# OpenApiToolkit

```{=mdx}

:::caution 免责声明 ⚠️

该代理可以向外部API发送请求。使用时请谨慎，尤其是在向用户授予访问权限时。

请注意，理论上该代理可能会通过提供的凭据或其他敏感数据向未验证或潜在恶意的URL发送请求——尽管它在理论上绝不应该这样做。

考虑添加限制，以控制代理可以通过哪些操作、可以访问哪些API、可以传递哪些请求头等。

另外，考虑实施措施，在发送请求之前验证URL，并安全地处理和保护敏感数据，如凭据。

:::

```

这将帮助您快速入门[OpenApiToolkit](/docs/concepts/tools/#toolkits)。如需查看所有OpenApiToolkit功能和配置的详细文档，请前往[API参考文档](https://api.js.langchain.com/classes/langchain.agents.OpenApiToolkit.html)。

`OpenAPIToolkit`可以访问以下工具：

| 名称            | 描述 |
|-----------------|-------------|
| `requests_get`  | 通往互联网的门户。当您需要从网站获取特定内容时使用。输入应该是一个URL字符串（例如"https://www.google.com"）。输出将是GET请求的文本响应。 |
| `requests_post` | 当您需要向网站POST数据时使用。输入应该是一个包含两个键的JSON字符串："url"和"data"。"url"的值应该是一个字符串，"data"的值应该是一个键值对字典，这些数据将作为JSON正文POST到指定的URL。请注意在JSON字符串中始终对字符串使用双引号。输出将是POST请求的文本响应。 |
| `json_explorer` | 可用于回答有关API的OpenAPI规范的问题。在尝试发出请求之前，请始终先使用此工具。此工具的示例输入：'对/bar端点执行GET请求所需的查询参数是什么？' '对/foo端点执行POST请求时请求体中需要哪些参数？' 请始终向此工具提出具体问题。 |

## 配置

该工具包需要一个OpenAPI规范文件。LangChain.js仓库在`examples`目录中提供了一个[示例OpenAPI规范文件](https://github.com/langchain-ai/langchainjs/blob/cc21aa29102571204f4443a40b53d28581a12e30/examples/openai_openapi.yaml)。您可以使用该文件测试工具包。

如果您希望获取各个工具运行的自动化追踪信息，还可以通过取消注释以下内容来设置您的[LangSmith](https://docs.smith.langchain.com/) API密钥：

```typescript
process.env.LANGSMITH_TRACING="true"
process.env.LANGSMITH_API_KEY="your-api-key"
```

### 安装

该工具包位于`langchain`包中：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  langchain @langchain/core
</Npm2Yarn>
```

## 实例化

现在我们可以实例化我们的工具包。首先，我们需要定义在工具包中要使用的LLM。

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [3]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai";
const llm = new ChatOpenAI({
  model: "gpt-4o-mini",
  temperature: 0,
})

In [4]:
import { OpenApiToolkit } from "langchain/agents/toolkits"
import * as fs from "fs";
import * as yaml from "js-yaml";
import { JsonSpec, JsonObject } from "langchain/tools";

// Load & convert the OpenAPI spec from YAML to JSON.
const yamlFile = fs.readFileSync("../../../../../examples/openai_openapi.yaml", "utf8");
const data = yaml.load(yamlFile) as JsonObject;
if (!data) {
  throw new Error("Failed to load OpenAPI spec");
}

// Define headers for the API requests.
const headers = {
  "Content-Type": "application/json",
  Authorization: `Bearer ${process.env.OPENAI_API_KEY}`,
};

const toolkit = new OpenApiToolkit(new JsonSpec(data), llm, headers);

## 工具

查看可用工具：

In [6]:
const tools = toolkit.getTools();

console.log(tools.map((tool) => ({
  name: tool.name,
  description: tool.description,
})))

[
  {
    name: 'requests_get',
    description: 'A portal to the internet. Use this when you need to get specific content from a website.\n' +
      '  Input should be a url string (i.e. "https://www.google.com"). The output will be the text response of the GET request.'
  },
  {
    name: 'requests_post',
    description: 'Use this when you want to POST to a website.\n' +
      '  Input should be a json string with two keys: "url" and "data".\n' +
      '  The value of "url" should be a string, and the value of "data" should be a dictionary of\n' +
      '  key-value pairs you want to POST to the url as a JSON body.\n' +
      '  Be careful to always use double quotes for strings in the json string\n' +
      '  The output will be the text response of the POST request.'
  },
  {
    name: 'json_explorer',
    description: '\n' +
      'Can be used to answer questions about the openapi spec for the API. Always use this tool before trying to make a request. \n' +
      'Example inputs 

## 在智能体中使用

首先，确保你已经安装了 LangGraph：

```{=mdx}
<Npm2Yarn>
  @langchain/langgraph
</Npm2Yarn>
```

In [5]:
import { createReactAgent } from "@langchain/langgraph/prebuilt"

const agentExecutor = createReactAgent({ llm, tools });

In [6]:
const exampleQuery = "Make a POST request to openai /chat/completions. The prompt should be 'tell me a joke.'. Ensure you use the model 'gpt-4o-mini'."

const events = await agentExecutor.stream(
  { messages: [["user", exampleQuery]]},
  { streamMode: "values", }
)

for await (const event of events) {
  const lastMsg = event.messages[event.messages.length - 1];
  if (lastMsg.tool_calls?.length) {
    console.dir(lastMsg.tool_calls, { depth: null });
  } else if (lastMsg.content) {
    console.log(lastMsg.content);
  }
}

[
  {
    name: 'requests_post',
    args: {
      input: '{"url":"https://api.openai.com/v1/chat/completions","data":{"model":"gpt-4o-mini","messages":[{"role":"user","content":"tell me a joke."}]}}'
    },
    type: 'tool_call',
    id: 'call_1HqyZrbYgKFwQRfAtsZA2uL5'
  }
]
{
  "id": "chatcmpl-9t36IIuRCs0WGMEy69HUqPcKvOc1w",
  "object": "chat.completion",
  "created": 1722906986,
  "model": "gpt-4o-mini-2024-07-18",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Why don't skeletons fight each other? \n\nThey don't have the guts!"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 12,
    "completion_tokens": 15,
    "total_tokens": 27
  },
  "system_fingerprint": "fp_48196bc67a"
}

Here's a joke for you:

**Why don't skeletons fight each other?**  
They don't have the guts!


## API 参考文档

如需了解所有 OpenApiToolkit 功能和配置的详细文档，请访问 [API 参考页面](https://api.js.langchain.com/classes/langchain.agents.OpenApiToolkit.html)。